<a href="https://colab.research.google.com/github/Nathan-Levy/Traffic-Flow-Prediction-in-Urban-Areas/blob/main/Compare_with_Baseline_PEMS03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the NPZ file
npz_file_path = "PEMS03.npz"
npz_data = np.load(npz_file_path)
traffic_data = npz_data['data']  # Shape: (26208, 358, 1)

# Reshape data to 2D (Flatten sensors to apply normalization)
num_timesteps, num_sensors, num_features = traffic_data.shape
traffic_data = traffic_data.reshape(num_timesteps, num_sensors)

# Normalize data using Min-Max Scaling (0 to 1)
scaler = MinMaxScaler()
traffic_data = scaler.fit_transform(traffic_data)

# Reshape back to original shape after normalization
traffic_data = traffic_data.reshape(num_timesteps, num_sensors, num_features)

# Function to create sequences (time-series sliding window)
def create_sequences(data, seq_length=12):
    """
    Convert the time series data into sliding window sequences.

    Parameters:
        data (numpy array): Normalized traffic flow data.
        seq_length (int): Number of past time steps to use as input.

    Returns:
        X (numpy array): Input sequences (features).
        y (numpy array): Target values (labels).
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Input: past `seq_length` time steps
        y.append(data[i+seq_length])    # Output: next time step
    return np.array(X), np.array(y)

# Define the sequence length (e.g., 12 past steps to predict next)
SEQ_LENGTH = 12
X, y = create_sequences(traffic_data, seq_length=SEQ_LENGTH)

# Remove the extra feature dimension if it is 1
X = X.squeeze(-1)  # Now X becomes (n_samples, SEQ_LENGTH, 358)
y = y.squeeze(-1)  # Now y becomes (n_samples, 358)


# Split into Train (80%) and Test (20%) sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Print dataset shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (20956, 12, 358), y_train shape: (20956, 358)
X_test shape: (5240, 12, 358), y_test shape: (5240, 358)


In [ ]:
autoencoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 12, 358)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 12, 200)             │         447,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 350)                 │         771,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 12, 350)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 12, 300)             │         781,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 12, 358)             │         107,758 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,322,676 (24.12 MB)

 Trainable params: 2,107,558 (8.04 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,215,118 (16.08 MB)

In [ ]:
from keras import layers
input_layer = keras.layers.Input(shape=(SEQ_LENGTH, 358))
encoded = keras.layers.LSTM(200, activation='tanh', return_sequences=True)(input_layer)
encoded = keras.layers.LSTM(350, activation='tanh', return_sequences=False)(encoded)


# Decoder
decoded = keras.layers.RepeatVector(SEQ_LENGTH)(encoded)
decoded = keras.layers.LSTM(300, activation='tanh', return_sequences=True)(decoded)
decoded = keras.layers.TimeDistributed(layers.Dense(358, activation='linear'))(decoded)

# Autoencoder Model
autoencoder = keras.models.Model(inputs=input_layer, outputs=decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 12, 358)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 12, 200)             │         447,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 350)                 │         771,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector_1 (RepeatVector)       │ (None, 12, 350)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 12, 300)             │         781,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 12, 358)             │         107,758 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,107,558 (8.04 MB)

 Trainable params: 2,107,558 (8.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train = autoencoder.fit(X_train, X_train, epochs=20, validation_data=(X_test, X_test))

Epoch 1/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.0130 - val_loss: 0.0038
Epoch 2/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.0029 - val_loss: 0.0033
Epoch 3/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 0.0026 - val_loss: 0.0031
Epoch 4/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 0.0023 - val_loss: 0.0029
Epoch 5/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 0.0022 - val_loss: 0.0028
Epoch 6/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 0.0021 - val_loss: 0.0028
Epoch 7/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0020 - val_loss: 0.0028
Epoch 8/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 0.0019 - val_loss: 0.0026
Epoch 9/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - loss: 0.0019 - val_loss: 0.0027
Epoch 10/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0019 - val_loss: 0.0028
Epoch 11/20
655/655 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0018 - val_loss: 0.0028
Epoch 12/20
655/655 ━━━━━━━━━━━━━━━━

In [ ]:
# Extract the encoder model
encoder = keras.models.Model(inputs=input_layer, outputs=encoded)

# Generate the encoded features
encoded_features = encoder.predict(X_train)
encoded_features_val = encoder.predict(X_test)

655/655 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
encoded_features = encoded_features.reshape(encoded_features.shape[0], 1, encoded_features.shape[1])
encoded_features_val = encoded_features_val.reshape(encoded_features_val.shape[0], 1, encoded_features_val.shape[1])

In [ ]:
# Define the prediction model
prediction_model = keras.models.Sequential()
prediction_model.add(keras.layers.LSTM(400, activation='relu', return_sequences=True, input_shape=encoded_features.shape[1:]))
prediction_model.add(keras.layers.LSTM(200, activation='relu', return_sequences=False))
prediction_model.add(keras.layers.Dense(358, activation='linear'))

# Compile the prediction model
prediction_model.compile(optimizer='adam', loss='mse')

# Train the prediction model
prediction_model.fit(encoded_features, y_train, epochs=5, validation_data=(encoded_features_val, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
655/655 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: 0.0327 - val_loss: 0.0038
Epoch 2/5
655/655 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0027 - val_loss: 0.0033
Epoch 3/5
655/655 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0024 - val_loss: 0.0031
Epoch 4/5
655/655 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0022 - val_loss: 0.0030
Epoch 5/5
655/655 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0021 - val_loss: 0.0030


In [ ]:
# Use the prediction model to make predictions
y_pred = prediction_model.predict(encoded_features_val)

164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [ ]:
# Reshape predictions and true values to 2D (samples, features)
y_pred_reshaped = y_pred.reshape(y_pred.shape[0], -1)  # Convert from (samples, sensors, 1) to (samples, sensors)
y_test_reshaped = y_test.reshape(y_test.shape[0], -1)

# Apply inverse transform
y_pred_rescaled = scaler.inverse_transform(y_pred_reshaped)
y_test_rescaled = scaler.inverse_transform(y_test_reshaped)


In [ ]:
import numpy as np

# Define evaluation metric functions
def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def mean_relative_error_filtered(y_true, y_pred, threshold=1e-3):
    mask = np.abs(y_true) > threshold
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / np.abs(y_true[mask]))


def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Function to evaluate model performance
def evaluate_model(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mre = mean_relative_error_filtered(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Relative Error (MRE): {mre:.4%}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


evaluate_model(np.array(y_test_rescaled), np.array(y_pred_rescaled))


Mean Absolute Error (MAE): 15.6726
Mean Relative Error (MRE): 15.8225%
Root Mean Squared Error (RMSE): 27.9058
